# Processamento de Linguagem Natural - Minicurso do SBBD 2022

Esse código foi desenvolvido para o minicurso de PLN no SBBD 2022.

Autoras: Helena Caseli, Cláudia Freitas e Roberta Viola 

https://sites.google.com/view/brasileiras-pln/

Fontes:
* Curso de Linguística Computacional da UFMG - Prof. Thiago Castro Ferreira
https://www.youtube.com/playlist?list=PLLrlHSmC0Mw73a1t73DEjgGMPyu8QssWT
* scikit-learn: https://scikit-learn.org/stable/
* SpaCy (https://spacy.io/)
* modelos do SpaCy para o português: https://spacy.io/models/pt
* NumPy: https://numpy.org/

# Corpus

In [ ]:
poema = ["E agora, José?", "A festa acabou,", "a luz apagou,", "o povo sumiu,", "a noite esfriou,", "e agora, José?", "e agora, você?", "você que é sem nome,", 
          "que zomba dos outros,", "você que faz versos,", "que ama, protesta?", "e agora, José?"]

# Pré-processamento com Spacy
(veja código em https://colab.research.google.com/drive/1117zJKfTFn5yHToWj0KRQG7WmSJw6IEa?usp=sharing para entender o básico)

In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy[cuda102]==3
!python3 -m spacy download pt_core_news_md

In [ ]:
import spacy

spacy.prefer_gpu()
pln = spacy.load("pt_core_news_md")


In [ ]:
sentenca = pln(poema[1])
tokens = []
for token in sentenca:
  tokens.append([token.text]) # Outras informações: token.lemma_, token.pos_, token.tag_, token.dep_, token.is_stop
print(tokens)

[['A'], ['festa'], ['acabou'], [',']]


# Representação *One-Hot*
Palavras e documentos são representados por vetores de dimensão do tamanho do vocabulário. Os vetores assumem valores binários (0 ou 1).

Vamos usar a biblioteca `OneHotEncoder` do Scikit Learn.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(tokens)

vocab= list(enc.categories_[0])
vetores = enc.transform(tokens).toarray()


Vetor One-Hot de *festa*

In [ ]:
print("Vetor one-hot que representa 'festa':") 
vetores[vocab.index('festa')]

Vetor one-hot que representa 'festa':


array([1., 0., 0., 0.])

# Matriz Frequência Termo-Documento
Dado um vocabulário e um conjunto de documentos, as representações das palavras e dos documentos podem ser calculadas a partir da contagem de cada palavra em cada documento.

Para isso vamos usar o método `CountVectorizer` do Scikit Learn.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Gerando a matriz de frequência termo-documento.

In [ ]:
vectorizer = CountVectorizer()

vetores = vectorizer.fit_transform(poema)
vocab = vectorizer.get_feature_names()

vetores.toarray(), vocab

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(array([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 ['acabou',
  'agora',
  'ama',
  'apagou',
  'dos',
  'esfriou',
  'faz',
  'festa',
  'josé',
  'luz',
  'noite',
  'nom

# Matriz de Frequência Termo-Termo

In [ ]:
import numpy as np

corpus_tok = [] 
for verso in poema:  
  corpus_tok.append(verso)

vocab = ['a', 'acabou', 'agora', 'ama', 'apagou', 'dos', 'e', 'esfriou', 'faz', 'festa', 'josé', 'luz',
  'noite', 'nome', 'o', 'outros', 'povo', 'protesta', 'que', 'sem', 'sumiu', 'versos', 'você', 'zomba', 'é']

vetores = np.zeros((len(vocab), len(vocab)))

for verso in corpus_tok:
  for i, w1 in enumerate(vocab):
    for j, w2 in enumerate(vocab):
      if i != j:
        if w1 in verso and w2 in verso:
          vetores[i, j] += 1

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(vetores)

Vocabulário
['a', 'acabou', 'agora', 'ama', 'apagou', 'dos', 'e', 'esfriou', 'faz', 'festa', 'josé', 'luz', 'noite', 'nome', 'o', 'outros', 'povo', 'protesta', 'que', 'sem', 'sumiu', 'versos', 'você', 'zomba', 'é']

Matrix
[[ 0.  1.  4.  1.  1.  1.  8.  1.  1.  1.  0.  1.  1.  0. 10.  1.  0.  1.
   3.  0.  0.  1.  2.  1.  3.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  4.  0.  0.  0.
   0.  0.  0.  0.  1.  0.  3.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.
   1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.
   1.  0.  0.  0.  0.  1.  0.]
 [ 8.  1.  3.  1.  0.  1.  0.  1.  1.  1.  0.  0.  1.  1.  9.  1.  0.  1.
   4.  1.  0.  1.  3.  1.  3.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  0.  0. 

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = Pipeline([('count', CountVectorizer()),
                 ('tfid', TfidfTransformer())])

vetores = vectorizer.fit_transform(poema)
vocab = vectorizer['count'].get_feature_names()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(np.round(vetores.toarray(), 2))

Vocabulário
['acabou', 'agora', 'ama', 'apagou', 'dos', 'esfriou', 'faz', 'festa', 'josé', 'luz', 'noite', 'nome', 'outros', 'povo', 'protesta', 'que', 'sem', 'sumiu', 'versos', 'você', 'zomba']

Matrix
[[0.   0.67 0.   0.   0.   0.   0.   0.   0.74 0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.  ]
 [0.71 0.   0.   0.   0.   0.   0.   0.71 0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.71 0.   0.   0.   0.   0.   0.71 0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.71
  0.   0.   0.   0.71 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.71 0.   0.   0.   0.   0.71 0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.67 0.   0.   0.   0.   0.   0.   0.74 0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.67 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.74 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Acessando o sexto ("e agora, José?") e o sétimo ("e agora, você?") versos e calculando a similaridade entre eles.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

verso6 = vetores[5, :]
verso7 = vetores[6, :]

cosine_similarity(verso6,verso7)[0][0]

0.4461813288112892

Fim deste código.